# Kaggle Inclass : Sequential
베이스라인

### Configurations

In [1]:
!pip install easydict pillow

  Created wheel for easydict: filename=easydict-1.9-py3-none-any.whl size=6350 sha256=96ee5ec801e9d5608c27bd663373a3fbd79fb9e1319232332c58ef5481cd7573
  Stored in directory: /root/.cache/pip/wheels/d3/e0/e9/305e348717e399665119bd012510d51ff4f22d709ff60c3096
Successfully built easydict


In [3]:
!pip install tensorboard

     |████████████████████████████████| 5.8 MB 1.0 MB/s eta 0:00:01    |▏                               | 30 kB 1.1 MB/s eta 0:00:06     |███▎                            | 583 kB 1.0 MB/s eta 0:00:06     |█████▉                          | 1.1 MB 1.0 MB/s eta 0:00:05     |████████████████████████        | 4.3 MB 1.0 MB/s eta 0:00:02
     |████████████████████████████████| 1.1 MB 24.7 MB/s eta 0:00:01
     |████████████████████████████████| 154 kB 34.9 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 4.5 MB/s  eta 0:00:01
     |████████████████████████████████| 3.9 MB 6.4 MB/s eta 0:00:01     |█                               | 112 kB 6.4 MB/s eta 0:00:01     |██████▍                         | 788 kB 6.4 MB/s eta 0:00:01
     |████████████████████████████████| 4.9 MB 38.1 MB/s eta 0:00:01
     |████████████████████████████████| 288 kB 40.3 MB/s eta 0:00:01
     |████████████████████████████████| 132 kB 34.7 MB/s eta 0:00:01
     |████████████████████████████████| 781 kB 27.7

In [1]:
import os
import sys
import math
import datetime
import numpy as np
import pandas as pd
import time
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
import argparse
import easydict
from torch import autograd

os.makedirs('log', exist_ok=True)

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('log/tensorboard')

In [2]:
torch.cuda.is_available()

True

In [3]:
random_seed = 2021
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
np.random.seed(random_seed)

In [4]:
DATASET_PATH = os.path.join('/USER/kaggle/traffic/data')

In [5]:
a = pd.read_csv(os.path.join(DATASET_PATH, 'train.csv'))
a

,날짜,시간,10,100,101,120,121,140,150,160,...,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
0,20200101,0,83247,19128,2611,5161,1588,892,32263,1636,...,1311,3482,11299,7072,1176,3810,748,3920,2133,3799
1,20200101,1,89309,19027,3337,5502,1650,1043,35609,1644,...,1162,3849,13180,8771,1283,3763,782,3483,2057,4010
2,20200101,2,66611,14710,2970,4631,1044,921,26821,1104,...,768,2299,7986,5426,1536,3229,491,2634,1526,3388
3,20200101,3,53290,13753,2270,4242,1021,790,21322,909,...,632,1716,5703,3156,1104,2882,431,2488,1268,3686
4,20200101,4,52095,17615,2406,3689,1840,922,22711,1354,...,875,2421,5816,2933,1206,2433,499,2952,1927,5608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3274,20200517,19,311727,101285,10085,30637,10060,8749,148935,6801,...,6726,15431,25597,14292,9300,22238,3786,16936,10729,20194
3275,20200517,20,305354,91426,8607,26021,8095,7198,136503,6147,...,5501,15378,24661,14747,8239,20604,3203,15018,9767,17962
3276,20200517,21,306008,75113,6325,19933,5711,4494,129412,5134,...,4216,12558,22781,14081,6392,17937,2447,12403,7825,14031
3277,20200517,22,237447,49498,4209,12145,3891,2718,96698,3526,...,2578,8870,16640,11066,4427,11955,1495,7507,5387,8889


In [7]:
pd.read_csv(os.path.join(DATASET_PATH, 'validate.csv'))

,날짜,시간,10,100,101,120,121,140,150,160,...,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
0,20200511,0,77968,14429,1233,4021,981,881,28672,1064,...,637,2604,5239,4168,1155,3596,337,2262,1608,2337
1,20200511,1,48679,9136,823,2618,654,572,17722,672,...,353,1870,3359,2558,1002,2157,257,1425,1018,1810
2,20200511,2,33773,8199,578,2188,392,502,14464,579,...,345,1499,2646,2022,876,1959,232,1155,927,1530
3,20200511,3,41511,9986,726,2817,555,646,17793,650,...,390,1730,3398,1967,912,2462,281,1477,959,1882
4,20200511,4,78680,19509,1463,4720,825,1088,35125,997,...,679,2958,7369,4120,1569,4568,577,3155,1871,3656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,20200524,19,314226,98345,10625,28618,8316,6684,141675,6619,...,8254,16118,23304,14082,8447,21694,2180,15746,10903,21014
332,20200524,20,300001,87871,8226,22706,6981,5743,142933,6295,...,5225,15297,21919,14526,7332,19732,1990,14096,10028,17787
333,20200524,21,304150,71126,6002,18317,4939,3779,133110,4781,...,4072,12685,21135,14403,5443,16967,1359,11670,7963,14041
334,20200524,22,236751,44947,3575,11455,3135,2536,98582,3267,...,2489,8093,14427,10914,3861,11397,859,7270,5194,8230


In [8]:
pd.read_csv(os.path.join(DATASET_PATH, 'test.csv'))

,날짜,시간,10,100,101,120,121,140,150,160,...,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
0,20200518,0,82065,15172,1500,3294,1086,962,28931,1103,...,618,2790,5147,4331,1329,3665,404,2242,1619,2314
1,20200518,1,51248,9840,813,2356,696,546,17888,720,...,430,1864,3269,2561,921,2081,272,1390,1003,1766
2,20200518,2,39026,7894,760,2413,408,549,13357,498,...,322,1313,2765,1931,920,1764,228,1136,922,1309
3,20200518,3,40993,10137,780,2701,420,741,15544,532,...,326,1766,3320,2060,892,2447,337,1495,975,1912
4,20200518,4,77863,19603,1276,5019,968,1160,32101,968,...,669,2914,6986,3911,1368,4380,513,2940,1758,3629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,20200531,19,-999,-999,-999,-999,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
332,20200531,20,-999,-999,-999,-999,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
333,20200531,21,-999,-999,-999,-999,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
334,20200531,22,-999,-999,-999,-999,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999


### Dataloader
* 한 칼럼에 대한 7일(168행) 데이터를 input_data, 뒤따르는 7일 데이터를 output_data로 반환합니다.
* 도로별 차이를 두지 않고 모든 도로를 동일한 타입의 데이터로 취급합니다.
* 모든 csv 파일의 마지막 168행은 예측해야하는 값이므로 input으로 들어가지 않습니다.

In [9]:
import torch
from torch.utils import data
from torchvision import datasets, transforms
import os


class CustomDataset(data.Dataset):
    def __init__(self, root, seq_len, batch_size=64, phase='train'):
        self.root = root
        self.phase = phase
        self.seq_len = seq_len * 24
        self.batch_size = batch_size
        self.labels = {}

        self.label_path = os.path.join(self.root, self.phase + '.csv')

        df = pd.read_csv(self.label_path)
        timestamps = [(i, j) for (i, j) in zip(list(df['날짜']), list(df['시간']))]
        categories = df.columns.values.tolist()[2:]

        input_data = []
        output_data = []

        for t in range(len(timestamps)):
            temp_input_data = []
            temp_output_data = []
            for col in categories:
                road = df[col].tolist()
                inp = [float(i) for i in road[t:t+self.seq_len]]
                outp = [float(j) for j in road[t+self.seq_len:t+2*self.seq_len]]
                temp_input_data.append(inp)
                temp_output_data.append(outp)
            input_data.append(temp_input_data)
            output_data.append(temp_output_data)

        self.labels['timestamp'] = timestamps
        self.labels['category'] = categories
        self.labels['input'] = input_data
        self.labels['output'] = output_data

    def __getitem__(self, index):

        row = index // 35
        col = index % 35

        timestamp = self.labels['timestamp'][row]
        category = self.labels['category'][col]
        
        input_data = torch.tensor(self.labels['input'][row][col])

        if self.phase != 'test':
            output_data = torch.tensor(self.labels['output'][row][col])
        else:
            output_data = []

        return timestamp, category, (input_data, output_data)

    def __len__(self):
        return (len(self.labels['timestamp']) - (self.seq_len * 2) + 1) * 35

    def get_label_file(self):
        return self.label_path


def data_loader(root, phase='train', batch_size=64, seq_len=7, drop_last=False):
    if phase == 'train':
        shuffle = True
    else:
        shuffle = False

    dataset = CustomDataset(root, seq_len, batch_size, phase)
    dataloader = data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last)

    return dataloader, dataset.get_label_file()

### Model

In [10]:
import torch
import torch.nn as nn
import torchvision.models as models

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class LSTMNet(nn.Module):
    def __init__(self,
                 input_size=168,
                 hidden_size=1024,
                 output_size=168,

                 batch_size=64,

                 num_layers=3,
                 dropout=0,
                 batch_first=False):
        super(LSTMNet, self).__init__()

        self.hidden_size = hidden_size
        
        ##### Layer 1
        self.lstm1 = nn.LSTM(input_size,
                            hidden_size,
                            dropout=0.2,
                            num_layers=num_layers)

        ##### Layer 2
        self.lstm2 = nn.LSTM(hidden_size, 
                             hidden_size,
                             dropout=0.2,
                             num_layers=num_layers)

        ##### Finalize
        self.linear = nn.Linear(hidden_size, 
                                output_size)
        
        self.activation = nn.LeakyReLU(0.2)


    def forward(self, x, h_in, c_in):

        h_in = nn.Parameter(h_in.type(dtype), requires_grad=True)
        c_in = nn.Parameter(c_in.type(dtype), requires_grad=True)

        # Layer 1
        lstm_out, (h_1, c_1) = self.lstm1(x, (h_in, c_in))
        lstm_out = self.activation(lstm_out)

        # Layer2
        lstm_out, (h_2, c_2) = self.lstm2(lstm_out, (h_1, c_1))
        lstm_out = self.activation(lstm_out)

        # Final
        predictions = self.linear(lstm_out)
        
        return predictions, (h_2, c_2)

In [11]:
class RMSLELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        
    def forward(self, pred, actual):
        return torch.sqrt(self.mse(torch.log(pred + 1), torch.log(actual + 1)))


def save_model(model_name, model, optimizer, scheduler):
    state = {
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'scheduler': scheduler.state_dict()
    }
    torch.save(state, os.path.join('log', model_name + '.pth'))
    print('model saved')
    return os.path.join('log', model_name + '.pth')


def load_model(model_name, model, optimizer=None, scheduler=None):
    state = torch.load(os.path.join(model_name))
    model.load_state_dict(state['model'])
    if optimizer is not None:
        optimizer.load_state_dict(state['optimizer'])
    if scheduler is not None:
        scheduler.load_state_dict(state['scheduler'])
    print('model loaded')

### Hyperparameters

In [12]:
dtype = torch.float
model_name = 'sequential'

batch_size = 64
num_epochs = 20
print_iter = 200
val_epoch = 1
save_epoch = 1
base_lr = 0.01
seq_len = 7

input_size = seq_len * 24
output_size = input_size
hidden_size = 1024
num_layers = 6

### Training Setting

결과 파일과 모델 가중치 파일 저장을 위해 log 디렉토리가 생성됩니다. 중요한 파일이 덮어씌워지지 않도록 주의 바랍니다.

In [14]:
os.makedirs('log', exist_ok=True)

In [13]:
# model
model = LSTMNet(input_size=input_size,
                hidden_size=hidden_size,
                output_size=output_size,
                batch_size=batch_size,
                num_layers=num_layers)
model = model.to(device)

# loss function
criterion = RMSLELoss()

# optimizer & scheduler
optimizer = Adam(model.parameters(), lr=base_lr)
scheduler = StepLR(optimizer, step_size=20, gamma=0.1)

In [ ]:
print(model)

In [14]:
# get data loader
train_dataloader, _ = data_loader(root=DATASET_PATH,
                                  phase='train',
                                  batch_size=batch_size,
                                  seq_len=seq_len,
                                  drop_last=True)

validate_dataloader, _ = data_loader(root=DATASET_PATH,
                                     phase='validate',
                                     batch_size=1,
                                     seq_len=seq_len,
                                     drop_last=True)

print("------------------------------------------------------------")
total_params = sum(p.numel() for p in model.parameters())
print("num of parameters : ",total_params)
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("num of trainable parameters :", trainable_params)
print("------------------------------------------------------------")

------------------------------------------------------------
num of parameters :  97427624
num of trainable parameters : 97427624
------------------------------------------------------------


### Train

In [17]:
running_loss = 0.0
epoch_loss, min_epoch_loss = 0.0, 99.0

for epoch in range(num_epochs):

    avg_loss = []
    model.train()

    for iter_, sample in enumerate(train_dataloader):

        (h_in, c_in) = (torch.zeros(num_layers, batch_size, hidden_size, requires_grad=True).to(device),
                        torch.zeros(num_layers, batch_size, hidden_size, requires_grad=True).to(device))

        _, _, (input_data, output_data) = sample
        input_data = input_data.unsqueeze(0).to(device)
        output_data = output_data.unsqueeze(0).to(device)
        print(input_data.size())
        pred, (h_in, c_in) = model(input_data, h_in, c_in)
        
        loss = criterion(pred, output_data)

        model.zero_grad()
        loss.backward()
        optimizer.step()

        avg_loss.append(loss.item())
        running_loss += loss.item()

        if iter_ % 200 == 199:
            writer.add_scalar('training loss',
                    running_loss / 200,
                    epoch * len(train_dataloader) + iter_)
            running_loss = 0

        if iter_ % print_iter == 0:
            print('Epoch: {:5} | Iteration: {:5} | Loss: {:1.5f}'.format(epoch, iter_, loss))

    scheduler.step()
    print('\nEpoch: {:5} | Loss: {:1.5f}\n'.format(epoch, sum(avg_loss) / len(avg_loss)))
    avg_loss = []
    
    if epoch_loss < min_epoch_loss:
        save_model('best', model, optimizer, scheduler)
        print(f'best model so far: epoch {epoch}')
        min_epoch_loss = epoch_loss

    if epoch % save_epoch == 0:
        save_model('last', model, optimizer, scheduler)
        print(f'last model: epoch {epoch}')

    if epoch % val_epoch == 0:
        
        val_score = 0.0
        model.eval()

        with torch.no_grad():
            (h_in, c_in) = (torch.zeros(num_layers, 1, hidden_size, requires_grad=False).to(device),
                    torch.zeros(num_layers, 1, hidden_size, requires_grad=False).to(device))

            for iter_, sample in enumerate(validate_dataloader):

                _, _, (input_data, output_data) = sample

                input_data = input_data.unsqueeze(0).to(device)
                output_data = output_data.unsqueeze(0).to(device)

                pred, (h_in, c_in) = model(input_data, h_in, c_in)
                score = criterion(pred, output_data)
                val_score += score.item()
        
        print('\nValidation Epoch: {:5} | Loss: {:1.5f}\n'.format(epoch, val_score/len(validate_dataloader)))
        model.train()

torch.Size([1, 64, 168])
Epoch:     0 | Iteration:     0 | Loss: 5.52821
torch.Size([1, 64, 168])
torch.Size([1, 64, 168])
torch.Size([1, 64, 168])
torch.Size([1, 64, 168])
torch.Size([1, 64, 168])
torch.Size([1, 64, 168])
torch.Size([1, 64, 168])
torch.Size([1, 64, 168])
torch.Size([1, 64, 168])
torch.Size([1, 64, 168])


KeyboardInterrupt: 

### Inference

In [18]:
dtype = torch.float
seq_len = 7

input_size = seq_len * 24
hidden_size = 1024
output_size = input_size
batch_size = 1
num_layers = 6

In [19]:
test_dataloader, _ = data_loader(root=DATASET_PATH,
                                  phase='test',
                                  batch_size=batch_size,
                                  seq_len=seq_len,
                                  drop_last=True)

In [20]:
model = LSTMNet(input_size=input_size,
                hidden_size=hidden_size,
                output_size=output_size,
                batch_size=batch_size,
                num_layers=num_layers)

# model
model_name = 'log/best.pth'

load_model(model_name, model)
model = model.to(device)

model loaded


In [21]:
submission_file_path = os.path.join(DATASET_PATH, 'sample_submission.csv')
submission_table = pd.read_csv(submission_file_path)

In [22]:
(h_in, c_in) = (torch.zeros(num_layers, 1, hidden_size, requires_grad=False).to(device),
                torch.zeros(num_layers, 1, hidden_size, requires_grad=False).to(device))

for iter_, sample in enumerate(test_dataloader):

    timestamp, category, (input_data, output_data) = sample
    input_data = input_data.unsqueeze(0).to(device)

    pred, (h_in, c_in) = model(input_data, h_in, c_in)

    for i, (t, h) in enumerate(zip(timestamp[0], timestamp[1])):
        for cat, row in zip(category, pred[0]):
            cat = f'road_{cat}'
            submission_table[cat] = row.tolist()

다음 셀을 실행하면 기존에 있던 prediction.csv가 덮어씌워집니다. 필요하시면 코드를 수정하거나 혹은 prediction.csv를 백업하시기 바랍니다.

In [23]:
submission_table.to_csv('prediction.csv', index=False)